<a href="https://colab.research.google.com/github/joaq-nucci/ManipulacaoBancoDados/blob/main/Desafio15_277177.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desafio 15**
Joaquim Bertoldi Nucci - RA: 277177

Esse código ilustra como Julia pode executar loops em série e em paralelo, distribuindo as tarefas entre múltiplos processos.
Ao usar addprocs, cria-se processos independentes capazes de efetuar cálculos simultaneamente.

A função criada multiplica grandes matrizes aleatórias, simulando carga computacional.
Compararemos o tempo gasto em série versus paralelizado com @distributed, calculando o speedup obtido.

In [1]:
# Instalação dos pacotes necessários
using Pkg
Pkg.add("BenchmarkTools")
Pkg.add("Distributed")

using BenchmarkTools
using Distributed

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed BenchmarkTools ─ v1.6.3
    Updating `~/.julia/environments/v1.11/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
  [9abbd945] + Profile v1.11.0
Precompiling project...
   3672.1 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 14 seconds. 492 already precompiled.
   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [8ba89e20] + Distributed v1.11.0
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [2]:
# Adiciona 4 processos workers
addprocs(4)
# Carrega o módulo em todos os workers
@everywhere using LinearAlgebra

In [3]:
# Função: multiplicação de matrizes grandes
@everywhere function tarefa_pesada(k)
    resultado = 0.0
    for i in 1:k
        A = rand(800, 800)
        B = rand(800, 800)
        resultado += sum(A * B)   # operação pesada
    end
    return resultado
end

In [4]:
# Execução em série
println("Executando em série...")
@time resultado_serie = [tarefa_pesada(10) for _ in 1:12]

# Execução em paralelo
println("\nExecutando em paralelo...")
@time resultado_paralelo = @distributed (vcat) for _ in 1:12
    tarefa_pesada(10)
end

Executando em série...
  6.594931 seconds (4.96 M allocations: 1.956 GiB, 5.09% gc time, 25.40% compilation time)

Executando em paralelo...
 18.437580 seconds (793.17 k allocations: 39.474 MiB, 1 lock conflict, 8.59% compilation time)


12-element Vector{Float64}:
 1.278933390851704e9
 1.2799490001679149e9
 1.27973477811941e9
 1.2804871636795425e9
 1.2799332645784726e9
 1.278854797357204e9
 1.279542924504149e9
 1.2799978828114629e9
 1.280284849012727e9
 1.280217128494059e9
 1.2796913962497075e9
 1.280309850770368e9

In [5]:
# Comparação detalhada
println("\n--- Comparação detalhada ---")

tempo_serie = @elapsed [tarefa_pesada(10) for _ in 1:12]
tempo_paralelo = @elapsed @distributed (vcat) for _ in 1:12
    tarefa_pesada(10)
end

println("Tempo em série: $(tempo_serie) segundos")
println("Tempo em paralelo: $(tempo_paralelo) segundos")
println("Speedup obtido: $(tempo_serie / tempo_paralelo)x")


--- Comparação detalhada ---
Tempo em série: 4.454801993 segundos
Tempo em paralelo: 4.62857596 segundos
Speedup obtido: 0.9624562784532978x


Os resultados mostram que o processamento paralelo foi um pouco mais lento que o sequencial. Enquanto a execução em série levou 4.45 segundos, a versão paralela levou 4.63 segundos, resultando em um speedup de apenas 0.96 vezes. Isso significa que não houve ganho de desempenho, na verdade, houve uma leve perda. Esse efeito ocorre porque, para tarefas relativamente leves, o custo de comunicação e coordenação entre processos supera os benefícios da paralelização.